In [1]:
import pandas as pd
import numpy as np

In [2]:
# Naming the matrix

techMat = 'data/tech_matrix.csv'# Technology Matrix
intervMat = 'data/intervMat.csv'#Intervention Matrix
fdemand1 = 'data/fdemand1.csv'#Final Demand with 1 MJ of corn ethanol

In [3]:
def csvToMatrix(fileName):
    fileName = pd.read_csv(fileName)
    fileName = pd.DataFrame(fileName).fillna(0)
    fileName = fileName.apply(pd.to_numeric, errors='coerce').dropna(axis='columns').to_numpy(dtype=float,na_value=0)
    return fileName

In [4]:
techMat=csvToMatrix(techMat)
intervMat=csvToMatrix(intervMat)
fdemand1 = pd.read_csv(fdemand1).values #turn fdemand table in csv format to an array

In [5]:
techMat.shape

(22, 22)

In [6]:
def calc_emission(technology_matrix, final_demand, intervention_matrix):
    '''
    calc_emission
    is used to calculate the emission from matrix.
    It takes three arguments, namely:
                        1) technology_matrix;
                        2) final_demand;
                        3) intervention_matrix
    '''
    technology_matrix_inv = np.linalg.inv(technology_matrix)
    scale_matrix = np.dot(technology_matrix_inv, final_demand)
    result_matrix = np.dot(intervention_matrix, scale_matrix)
    np.savetxt("data/result.csv", result_matrix, delimiter=",")
    print (result_matrix)

In [7]:
calc_emission(techMat, fdemand1, intervMat)

[[-1.07543671e+28]
 [-1.52325440e+28]
 [-2.41525548e+29]]


In [8]:
# Calculating the emission of each contributor and total here
result = pd.read_csv('data/result.csv', names=['kg'])# Technology Matrix
emission = result['kg']
CO2=emission[0] # 1st Row is CO2
CH4=emission[1] # 2nd Row is CH4
N2O=emission[2] # 3rd Row is N2O
CO2eq_CH4 = CH4 * 28  # CO2-eq from CH4, CH4*28
CO2eq_N2O = N2O * 265 # CO2-eq from N2O, CH4*265
CO2_tot_kggal = CO2 + CO2eq_CH4 +CO2eq_N2O
MJ_per_gal = 89 # 1 gallon equals 89 MJ of ethanol, 
                    # https://www.ocean.washington.edu/courses/envir215/energynumbers.pdf
kg_to_g = 1000 # converting kg to g, the original emission is in kg
CO2_tot_gPerMJ = CO2_tot_kggal * kg_to_g / MJ_per_gal

In [9]:
# Making tabulated result ---> save to csv
CO2_CO2e_gPerMJ = CO2 * kg_to_g / MJ_per_gal
CH4_CO2e_gPerMJ = CO2eq_CH4 * kg_to_g / MJ_per_gal
N2O_CO2e_gPerMJ = CO2eq_N2O * kg_to_g / MJ_per_gal

# Total/Accumulative
kg_total = CO2 + CH4 + N2O
CO2eq_kgPerGal_total = CO2 + CO2eq_CH4 + CO2eq_N2O
CO2e_gPerMJ_total = CO2_CO2e_gPerMJ + CH4_CO2e_gPerMJ + N2O_CO2e_gPerMJ

headerList = [ "kg", "kg CO2e/gal", "g CO2e/MJ"]
emissionList = ["CO2", "CH4", "N2O", "TOTAL"]
data = np.array([[CO2,CO2, CO2_CO2e_gPerMJ],
                [CH4,CO2eq_CH4, CH4_CO2e_gPerMJ],
                [N2O,CO2eq_N2O, N2O_CO2e_gPerMJ],
                [kg_total, CO2eq_kgPerGal_total, CO2e_gPerMJ_total]])

data_result = pd.DataFrame(data, emissionList, headerList )
np.savetxt("data/data_result.csv", data_result, delimiter=",")
print('The tabulated results are as following:')
print('---------------------------------------')
print(data_result)
print('---------------------------------------')
print (f'The total CO2 equivalent obtained from this calculation is {CO2_tot_kggal:.6} kg CO2e/gallon ethanol.')
print (f'With all necessary unit conversions, the CO2 equivalent amount is {CO2_tot_gPerMJ:.6} g CO2e/MJ ethanol.')

The tabulated results are as following:
---------------------------------------
                 kg   kg CO2e/gal     g CO2e/MJ
CO2   -1.075437e+28 -1.075437e+28 -1.208356e+29
CH4   -1.523254e+28 -4.265112e+29 -4.792261e+30
N2O   -2.415255e+29 -6.400427e+31 -7.191491e+32
TOTAL -2.675125e+29 -6.444154e+31 -7.240622e+32
---------------------------------------
The total CO2 equivalent obtained from this calculation is -6.44415e+31 kg CO2e/gallon ethanol.
With all necessary unit conversions, the CO2 equivalent amount is -7.24062e+32 g CO2e/MJ ethanol.


In [10]:
#Based on Prof. Zhao's announcement, per MJ of ethanol
CO2_ref = 2.9/89 #kg;
CH4_ref = 0.024/89 #kg;
N2O_ref = 0.0011/89 #kg;

In [11]:
# Result Error comparison
CO2_error = (CO2_ref-CO2)/CO2_ref*100
CH4_error = (CH4_ref-CH4)/CH4_ref*100
N2O_error = (N2O_ref-N2O)/N2O_ref*100

In [12]:
headerList = [ "Reference", "Results", "Error %"]
emissionList = ["CO2", "CH4", "N2O"]
data = np.array([[CO2_ref, CO2, CO2_error],
                [CH4_ref, CH4, CH4_error],
                [N2O_ref, N2O, N2O_ref]])

error_result = pd.DataFrame(data, emissionList, headerList )
np.savetxt("data/error_result.csv", error_result, delimiter=",")
print('The Results Error are as following:')
print('---------------------------------------')
print(error_result)


The Results Error are as following:
---------------------------------------
     Reference       Results       Error %
CO2   0.032584 -1.075437e+28  3.300478e+31
CH4   0.000270 -1.523254e+28  5.648735e+33
N2O   0.000012 -2.415255e+29  1.235955e-05


In [13]:
# Result Error comparison version 2
# If the result is bigger than the reference

CO2_error2 = (CO2-CO2_ref)/CO2*100
CH4_error2 = (CH4-CH4_ref)/CH4*100
N2O_error2 = (N2O-N2O_ref)/N2O*100

In [14]:
headerList = [ "Reference", "Results", "Error %"]
emissionList = ["CO2", "CH4", "N2O"]
data = np.array([[CO2_ref, CO2, CO2_error2],
                [CH4_ref, CH4, CH4_error2],
                [N2O_ref, N2O, N2O_error2]])

error_result2 = pd.DataFrame(data, emissionList, headerList )
np.savetxt("data/error_result2.csv", error_result2, delimiter=",")
print('The Results Error are as following:')
print('---------------------------------------')
print(error_result2)



The Results Error are as following:
---------------------------------------
     Reference       Results  Error %
CO2   0.032584 -1.075437e+28    100.0
CH4   0.000270 -1.523254e+28    100.0
N2O   0.000012 -2.415255e+29    100.0
